# Step 0 - Set up the Cluster

In this phase, we will provision and configure the cluster that will be used in the experiment. Before setting up the cluster, we will also set up an Azure File Share to host the executable and the script files. Begin by importing some prerequisite libraries. 

In [1]:
#Standard python libraries
import json
import os
import re
import datetime
import time

from IPython.display import clear_output

#Azure file storage. To install, run 'pip install azure-storage-file'
from azure.storage.file import FileService
from azure.storage.file import ContentSettings

#Azure blob. To install, run 'pip install azure-storage-blob'
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

#Azure batch. To install, run 'pip install cryptography azure-batch azure-storage'
import azure.storage.blob as azureblob
import azure.batch.models as batchmodels
import azure.batch.batch_auth as batchauth
import azure.batch as batch

with open('notebook_config.json', 'r') as f:
    NOTEBOOK_CONFIG = json.loads(f.read()) 

First, we will generate some prerequisite files. These files are used during the setup process to configure the machine. They require information unique to your cluster (such as account keys), so we will need to generate them from a template. The three files are:

* **mount.bat**: This batch file mounts an azure file share to a machine. It will mount the specified file share to the Z:\\ directory
* **run_airsim_on_user_login.xml**: This XML defines a scheduled task that will restart the AirSim simulator when a user logs into an agent node. This is necessary because Azure Batch starts the executable in session 0, which means that the simulator will be accessible via API, but not visible on the screen. By restarting it on login, we can visualize the training process.
* **setup_machine.py**: This script installs the prerequisite python libraries and configures the machine to properly run AirSim. 

In [6]:
#Generate mount.bat
with open('Template\\mount_bat.template', 'r') as f:
    mount_bat_cmd = f.read()
    
mount_bat_cmd = mount_bat_cmd\
                    .replace('{storage_account_name}', NOTEBOOK_CONFIG['storage_account_name'])\
                    .replace('{file_share_name}', NOTEBOOK_CONFIG['file_share_name'])\
                    .replace('{storage_account_key}', NOTEBOOK_CONFIG['storage_account_key'])

with open('Blob\\mount.bat', 'w') as f:
    f.write(mount_bat_cmd)
    
#Generate setup_machine.py
with open('Template\\setup_machine_py.template', 'r') as f:
    setup_machine_py = f.read()

setup_machine_py = setup_machine_py\
                    .replace('{storage_account_name}', NOTEBOOK_CONFIG['storage_account_name'])\
                    .replace('{file_share_name}', NOTEBOOK_CONFIG['file_share_name'])\
                    .replace('{storage_account_key}', NOTEBOOK_CONFIG['storage_account_key'])\
                    .replace('{batch_job_user_name}', NOTEBOOK_CONFIG['batch_job_user_name'])\
                    .replace('{batch_job_user_password}', NOTEBOOK_CONFIG['batch_job_user_password'])

with open('Blob\\setup_machine.py', 'w') as f:
    f.write(setup_machine_py)
    
#Generate run_airsim_on_user_login.xml
with open('Template\\run_airsim_on_user_login_xml.template', 'r', encoding='utf-16') as f:
    startup_task_xml = f.read()
    
startup_task_xml = startup_task_xml\
                    .replace('{batch_job_user_name}', NOTEBOOK_CONFIG['batch_job_user_name'])

with open('Share\\scripts_downpour\\run_airsim_on_user_login.xml', 'w', encoding='utf-16') as f:
    f.write(startup_task_xml)    

Now that we have all of the prerequisite files generated, the next step is to create the Azure File Share. We create the file share and upload all of the files under the /Share directory of the downloaded data files. Inside this directory, there are two folders:

* **AirsimOpenGL**: This folder contains the AirSim executable, as well as some helper scripts used to launch the exe.
* **scripts_downpour**: This folder contains the actual scripts that will be executed during the batch job. For more information about these scripts, see Step 1 - Explore the Algorithm.

In [25]:
file_service = FileService(account_name = NOTEBOOK_CONFIG['storage_account_name'], account_key=NOTEBOOK_CONFIG['storage_account_key'])
file_service.create_share(NOTEBOOK_CONFIG['file_share_name'], fail_on_exist=False)

True

Upload all files to share.

In [26]:
def create_directories(path, file_service):
    split_dir = path.split('\\')
    for i in range(1, len(split_dir)+1, 1):
        combined_dir = '\\'.join(split_dir[:i])
        file_service.create_directory(NOTEBOOK_CONFIG['file_share_name'], combined_dir, fail_on_exist=False)

for root, directories, files in os.walk('Share'):
    for file in files:
        regex_pattern = '{0}[\\\\]?'.format('Share').replace('\\', '\\\\')
        upload_directory = re.sub(regex_pattern, '', root)
        print('Uploading {0} to {1}...'.format(os.path.join(root, file), upload_directory))
        if (len(upload_directory) == 0):
            upload_directory = None
        if (upload_directory != None):
            create_directories(upload_directory, file_service)
        file_service.create_file_from_path(          
            NOTEBOOK_CONFIG['file_share_name'], 
            upload_directory,                   
            file,                               
            os.path.join(root, file)            
            )
        
        

Uploading Share\data\pretrain_model_weights.h5 to data...


Client-Request-ID=82f9c662-1511-11e8-a967-40167e77d4b0 Retry policy did not allow for a retry: Server-Timestamp=Mon, 19 Feb 2018 01:09:19 GMT, Server-Request-ID=aaf1c07d-001a-007a-451e-a9509f000000, HTTP status code=409, Exception=The specified resource already exists.<?xml version="1.0" encoding="utf-8"?><Error><Code>ResourceAlreadyExists</Code><Message>The specified resource already exists.RequestId:aaf1c07d-001a-007a-451e-a9509f000000Time:2018-02-19T01:09:19.3796423Z</Message></Error>.


Uploading Share\data\reward_points.txt to data...


Client-Request-ID=8322d2ca-1511-11e8-a160-40167e77d4b0 Retry policy did not allow for a retry: Server-Timestamp=Mon, 19 Feb 2018 01:09:19 GMT, Server-Request-ID=aaf1c080-001a-007a-481e-a9509f000000, HTTP status code=409, Exception=The specified resource already exists.<?xml version="1.0" encoding="utf-8"?><Error><Code>ResourceAlreadyExists</Code><Message>The specified resource already exists.RequestId:aaf1c080-001a-007a-481e-a9509f000000Time:2018-02-19T01:09:19.6488217Z</Message></Error>.


Uploading Share\data\road_lines.txt to data...
Uploading Share\scripts_downpour\manage.py to scripts_downpour...


...

Uploading Share\tools\x64\7za.dll to tools\x64...


Client-Request-ID=9354917e-1511-11e8-8682-40167e77d4b0 Retry policy did not allow for a retry: Server-Timestamp=Mon, 19 Feb 2018 01:09:46 GMT, Server-Request-ID=aaf1c137-001a-007a-5c1e-a9509f000000, HTTP status code=409, Exception=The specified resource already exists.<?xml version="1.0" encoding="utf-8"?><Error><Code>ResourceAlreadyExists</Code><Message>The specified resource already exists.RequestId:aaf1c137-001a-007a-5c1e-a9509f000000Time:2018-02-19T01:09:46.8180761Z</Message></Error>.
Client-Request-ID=9361d81e-1511-11e8-8b7d-40167e77d4b0 Retry policy did not allow for a retry: Server-Timestamp=Mon, 19 Feb 2018 01:09:46 GMT, Server-Request-ID=aaf1c138-001a-007a-5d1e-a9509f000000, HTTP status code=409, Exception=The specified resource already exists.<?xml version="1.0" encoding="utf-8"?><Error><Code>ResourceAlreadyExists</Code><Message>The specified resource already exists.RequestId:aaf1c138-001a-007a-5d1e-a9509f000000Time:2018-02-19T01:09:46.9051347Z</Message></Error>.


Uploading Share\tools\x64\7za.exe to tools\x64...


Client-Request-ID=9576680c-1511-11e8-947f-40167e77d4b0 Retry policy did not allow for a retry: Server-Timestamp=Mon, 19 Feb 2018 01:09:50 GMT, Server-Request-ID=aaf1c141-001a-007a-621e-a9509f000000, HTTP status code=409, Exception=The specified resource already exists.<?xml version="1.0" encoding="utf-8"?><Error><Code>ResourceAlreadyExists</Code><Message>The specified resource already exists.RequestId:aaf1c141-001a-007a-621e-a9509f000000Time:2018-02-19T01:09:50.3944838Z</Message></Error>.
Client-Request-ID=9583aeae-1511-11e8-9547-40167e77d4b0 Retry policy did not allow for a retry: Server-Timestamp=Mon, 19 Feb 2018 01:09:50 GMT, Server-Request-ID=aaf1c142-001a-007a-631e-a9509f000000, HTTP status code=409, Exception=The specified resource already exists.<?xml version="1.0" encoding="utf-8"?><Error><Code>ResourceAlreadyExists</Code><Message>The specified resource already exists.RequestId:aaf1c142-001a-007a-631e-a9509f000000Time:2018-02-19T01:09:50.4815419Z</Message></Error>.


Uploading Share\tools\x64\7zxa.dll to tools\x64...


When provisioning the machines for the Azure Batch pool, it is necessary to pull some of the setup scripts from blob. So in this step, we upload these prerequisite files to the blob storage. 

In [12]:
block_blob_service = BlockBlobService(account_name = NOTEBOOK_CONFIG['storage_account_name'], account_key = NOTEBOOK_CONFIG['storage_account_key'])
block_blob_service.create_container('prereq', public_access = PublicAccess.Container)

for root, directories, files in os.walk('Blob'):
    for file in files:
        block_blob_service.create_blob_from_path( 
            'prereq',                             
            file,                                 
            os.path.join(root, file)              
            )

We have a custom image that has the proper drivers installed to run AirSim. To create this image, we will run a powershell script that will copy the image from our storage account to your storage account. Ensure that you have the latest version of the [AzCopy utility](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy) installed and in your path (i.e. running 'azcopy' at the command line should yield the help page). In addition, ensure that you have the latest version of [Azure Powershell](https://docs.microsoft.com/en-us/powershell/azure/install-azurerm-ps?view=azurermps-5.3.0) installed. This command can take up to an hour to run.

In [16]:
os.system('powershell.exe ".\\CreateImage.ps1 -subscriptionId {0} -storageAccountName {1} -storageAccountKey {2} -resourceGroupName {3}'\
          .format(NOTEBOOK_CONFIG['subscription_id'], NOTEBOOK_CONFIG['storage_account_name'], NOTEBOOK_CONFIG['storage_account_key'], NOTEBOOK_CONFIG['resource_group_name']))

0

Finally, we create the pool of machines that will run our experiment. The important aspects of the machine configuration are:

* **image_reference**: We specify the data science VM to ensure that we have the correct drivers installed that will allow us to utilize the GPU.
* **vm_size**: The AirSim executable will only run on NV machines, so we choose the NV6 VMs.
* **target_dedicated_nodes**: The number of nodes to provision for the cluster. Note that 1 node will become your trainer, and the rest will become the agent. Ensure that there are enough cores available in your batch account to provision the number of VMs you are requesting - for example, the NV6 machines utilize 4 cores for each machine provisioned.
* **enable_inter_node_communication**: This parameter will allow the nodes to communicate with each other. Enabling this parameter limits the number of nodes to 40.
* **user_accounts**: We define an admin user to run the batch jobs. This user will also be used to log into the VMs and visualize the progress
* **start_task**: This is the task that will be run when the machines are being provisioned. In this phase, we download the prereq scripts and run them in the python environment. This will install the necessary python libraries and configure the machine to use AirSim.

In [2]:
batch_credentials = batchauth.SharedKeyCredentials(NOTEBOOK_CONFIG['batch_account_name'], NOTEBOOK_CONFIG['batch_account_key'])
batch_client = batch.BatchServiceClient(batch_credentials, base_url=NOTEBOOK_CONFIG['batch_account_url'])

#image_ref_to_use = batchmodels.ImageReference(publisher='microsoft-ads', offer='standard-data-science-vm', sku='standard-data-science-vm')
#node_agent_sku_to_use = 'batch.node.windows amd64'

image_path = '/subscriptions/{0}/resourceGroups/{1}/providers/Microsoft.Compute/images/{2}'\
                    .format(NOTEBOOK_CONFIG['subscription_id'], NOTEBOOK_CONFIG['resource_group_name'], 'AirsimImage3')
    
print(image_path)

image_ref_to_use = batchmodels.ImageReference(virtual_machine_image_id=image_path)
node_agent_sku_to_use = 'batch.node.windows amd64'

new_pool = batchmodels.PoolAddParameter(
    id=NOTEBOOK_CONFIG['batch_pool_name'],
    virtual_machine_configuration=batchmodels.VirtualMachineConfiguration(
        image_reference=image_ref_to_use,
        node_agent_sku_id=node_agent_sku_to_use),
    vm_size='STANDARD_NV6',
    target_dedicated_nodes=NOTEBOOK_CONFIG['batch_pool_size'],
    enable_inter_node_communication=True,
    user_accounts = [batchmodels.UserAccount(name=NOTEBOOK_CONFIG['batch_job_user_name'], password=NOTEBOOK_CONFIG['batch_job_user_password'], elevation_level='admin')],
    start_task=batchmodels.StartTask(
        command_line='C:\\ProgramData\\Anaconda3\\Scripts\\activate.bat py36 && python C:\\prereq\\setup_machine.py',
        wait_for_success=True,
        user_identity= batchmodels.UserIdentity(user_name=NOTEBOOK_CONFIG['batch_job_user_name']),
         resource_files= [
             batchmodels.ResourceFile(
                 blob_source='https://{0}.blob.core.windows.net/prereq/setup_machine.py'.format(NOTEBOOK_CONFIG['storage_account_name']),
                 file_path='C:\\prereq\\setup_machine.py'
             ),
              batchmodels.ResourceFile(
                 blob_source='https://{0}.blob.core.windows.net/prereq/mount.bat'.format(NOTEBOOK_CONFIG['storage_account_name']),
                 file_path='C:\\prereq\\mount.bat'
             )
         ]
    )
)

batch_client.pool.add(new_pool)

/subscriptions/ba86c6da-f243-4ab1-8158-ae8db3969df1/resourceGroups/mspryn_rg/providers/Microsoft.Compute/images/AirsimImage3
query_parameters
{'api-version': '2017-09-01.6.0'}
Headers:
{'Content-Type': 'application/json; odata=minimalmetadata; charset=utf-8', 'client-request-id': 'ce46bc1c-156b-11e8-a312-40167e77d4b0', 'accept-language': 'en-US'}
Body content: 
{'id': 'mspryn_pool_2', 'vmSize': 'STANDARD_NV6', 'virtualMachineConfiguration': {'imageReference': {'virtualMachineImageId': '/subscriptions/ba86c6da-f243-4ab1-8158-ae8db3969df1/resourceGroups/mspryn_rg/providers/Microsoft.Compute/images/AirsimImage3'}, 'nodeAgentSKUId': 'batch.node.windows amd64'}, 'targetDedicatedNodes': 1, 'enableInterNodeCommunication': True, 'startTask': {'commandLine': 'C:\\ProgramData\\Anaconda3\\Scripts\\activate.bat py36 && python C:\\prereq\\setup_machine.py', 'resourceFiles': [{'blobSource': 'https://msprynrrnotebook2.blob.core.windows.net/prereq/setup_machine.py', 'filePath': 'C:\\prereq\\setup_mach

BatchErrorException: {'lang': 'en-US', 'value': 'Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.\nRequestId:5fb55d53-c0e4-43fe-b140-715dd6f0d37a\nTime:2018-02-19T11:55:40.1139214Z'}

Alternatively, here's how to do it with the CLI. Note that there will be a code that you need to login output to your terminal window.

In [3]:
with open('Template\\pool.json.template', 'r') as f:
    pool_config = f.read()
    
pool_config = pool_config\
                .replace('{batch_pool_name}', NOTEBOOK_CONFIG['batch_pool_name'])\
                .replace('{subscription_id}', NOTEBOOK_CONFIG['subscription_id'])\
                .replace('{resource_group_name}', NOTEBOOK_CONFIG['resource_group_name'])\
                .replace('{storage_account_name}', NOTEBOOK_CONFIG['storage_account_name'])\
                .replace('{batch_job_user_name}', NOTEBOOK_CONFIG['batch_job_user_name'])\
                .replace('{batch_job_user_password}', NOTEBOOK_CONFIG['batch_job_user_password'])\
                .replace('{batch_pool_size}', str(NOTEBOOK_CONFIG['batch_pool_size']))

with open('pool.json', 'w') as f:
    f.write(pool_config)
    
create_cmd = 'powershell.exe ".\ProvisionCluster.ps1 -subscriptionId {0} -resourceGroupName {1} -batchAccountName {2}"'\
    .format(NOTEBOOK_CONFIG['subscription_id'], NOTEBOOK_CONFIG['resource_group_name'], NOTEBOOK_CONFIG['batch_account_name'])
    
print('Executing command. Check the terminal output for authentication instructions.')

os.system(create_cmd)

Executing command. Check the terminal output for authentication instructions.


0

Once this task finishes, you should see the pool created in your batch account, and you are ready to move on to **Step 1 - Explore the Algorithm** 